In [ ]:
# GraphRAG 설치
!pip install graphrag

!pip install pandas pyarrow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.0/401.0 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.6/388.6 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.3/191.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.0/429.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### GraphRAG 루트 및 API key 설정

In [ ]:
import os
import pandas as pd

PROJECT_ROOT = "   "
INPUT_DIR = os.path.join(PROJECT_ROOT, 'input')
os.makedirs(INPUT_DIR, exist_ok=True)

In [ ]:
import os
from google.colab import userdata

try:
    os.environ["GRAPHRAG_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("API Key가 성공적으로 로드되었습니다.")
except Exception as e:
    print("키 로드 실패: 키를 등록했는지 확인하세요.")

API Key가 성공적으로 로드되었습니다.


In [ ]:
# 처음 한번만 필요

!python -m graphrag init --root " "


'\n!python -m graphrag init --root "/content/drive/MyDrive/Colab Notebooks/NLP_EWAH/GraphRAG"\n'

In [ ]:
# 처음 한번만 필요

import subprocess

# settings.yaml, .env 생성
subprocess.run(
    ["graphrag", "init", "--root", str(PROJECT_ROOT)],
    check=True
)


'\nimport subprocess\n\n# settings.yaml, .env 생성\nsubprocess.run(\n    ["graphrag", "init", "--root", str(PROJECT_ROOT)],\n    check=True\n)\n'

### settings.yaml 수정 (models / input 섹션)

In [ ]:
import yaml

In [ ]:
settings_path = os.path.join( PROJECT_ROOT, "settings.yaml")
with open(settings_path, 'r') as f:
    settings = yaml.safe_load(f)

In [ ]:
# 1) 모델 설정 수정: gpt-4o-mini + text-embedding-3-small
#   - default_chat_model
chat_model = settings.setdefault("models", {}).setdefault("default_chat_model", {})
chat_model.setdefault("type", "openai_chat")
chat_model["api_key"] = "${GRAPHRAG_API_KEY}"
chat_model["model"] = "gpt-4o-mini"
chat_model["model_supports_json"] = False

#  Tier 3 속도 제한 해제 (기본값은 매우 낮음)
chat_model["requests_per_minute"] = 10_000
chat_model["tokens_per_minute"] = 2_000_000
chat_model["max_retries"] = 3
chat_model["concurrent_requests"] = 30

#   - default_embedding_model
embed_model = settings["models"].setdefault("default_embedding_model", {})
embed_model.setdefault("type", "openai_embedding")
embed_model["api_key"] = "${GRAPHRAG_API_KEY}"
embed_model["model"] = "text-embedding-3-small"

# 임베딩 모델도 속도 제한을 풀어줍니다
embed_model["requests_per_minute"] = 10_000
embed_model["tokens_per_minute"] = 2_000_000

parallel_cfg = settings.setdefault("parallelization", {})
parallel_cfg["stagger"] = 0.1
parallel_cfg["num_threads"] = 50

# 2) input 설정: corpus.csv 를 읽도록 구성
input_cfg = settings.setdefault("input", {})
input_cfg["type"] = "file"
input_cfg["file_type"] = "csv"
input_cfg["base_dir"] = "input"      # PROJECT_ROOT/input 아래를 보게 됨
input_cfg["text_column"] = "text"
input_cfg["title_column"] = "title"
# source_id 를 metadata 로 보존
input_cfg["metadata"] = ["source_id"]

# 3) chunking 기본값 (원하는 대로 조절 가능)
chunks_cfg = settings.setdefault("chunks", {})
chunks_cfg.setdefault("size", 1000)      # chunking 설정
chunks_cfg.setdefault("overlap", 100)    # overlap 설정
chunks_cfg.setdefault("strategy", "tokens")
chunks_cfg.setdefault("prepend_metadata", False)
chunks_cfg.setdefault("chunk_size_includes_metadata", False)

# 4) output / cache 기본 위치 (루트 기준 상대경로)
output_cfg = settings.setdefault("output", {})
output_cfg.setdefault("type", "file")
output_cfg.setdefault("base_dir", "output")

cache_cfg = settings.setdefault("cache", {})
cache_cfg.setdefault("type", "file")
cache_cfg.setdefault("base_dir", "cache")

snap_cfg = settings.setdefault("snapshots", {})
snap_cfg["graphml"] = False
snap_cfg["raw_entities"] = False
snap_cfg["top_level_nodes"] = False

claims_cfg = settings.setdefault("claim_extraction", {})
claims_cfg["enabled"] = False
claims_cfg["prompt"] = "prompts/claim_extraction.txt" # 파일이 없어도 false면 무시됨
claims_cfg["description"] = "Any claims or facts that are extracted from the text"
claims_cfg["max_gleanings"] = 1

with open(settings_path, "w") as f:
    yaml.safe_dump(settings, f, sort_keys=False)

print("settings.yaml updated for gpt-4o-mini + text-embedding-3-small")

settings.yaml updated for gpt-4o-mini + text-embedding-3-small


#### corpus.parquet → CSV 변환 (GraphRAG input schema)

In [ ]:
CORPUS_PATH = '  '
corpus = pd.read_parquet(CORPUS_PATH)


In [ ]:
print(corpus.columns)
print(corpus.shape)

Index(['title', 'text', 'source_id'], dtype='object')
(2000, 3)


In [ ]:
corpus_csv_path =os.path.join( INPUT_DIR , "corpus_validation.csv")
corpus.to_csv(corpus_csv_path, index = False)

GraphRAG 인덱싱 실행

In [ ]:
import subprocess

# 명령어 리스트 구성
cmd = [
    "graphrag",
    "index",
    "--root", str(PROJECT_ROOT),
    "--config", str(settings_path),
    "--method", "fast",
    "--cache",
]

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)


full_output = []


for line in process.stdout:
    print(line, end='')
    full_output.append(line)


process.wait()

print(f"\n종료 코드 (Return code): {process.returncode}")


🚀 GraphRAG 인덱싱을 시작합니다... (실시간 진행 상황 출력)
2025-12-06 15:04:59.838150: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765033499.860012    2922 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765033499.866665    2922 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765033499.883217    2922 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765033499.883240    2922 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765033499.883242    2922 computati

In [ ]:
print(os.listdir(PROJECT_ROOT))

['input', '.env', 'prompts', 'logs', 'output', 'cache', 'settings.yaml']


In [ ]:
print(os.listdir(os.path.join(PROJECT_ROOT, 'output')))

['lancedb', 'context.json', 'documents.parquet', 'stats.json', 'entities.parquet', 'communities.parquet', 'relationships.parquet', 'text_units.parquet', 'community_reports.parquet']
